# Data Conversion Challenge
Challenge to automate the conversion of raw data into a specified format of data to make it more usable.

**Important note**: The data used in this notebook has been randomised and all names have been masked so they can be used for training purposes. This notebook is for training purposes only.

This notebook is available in the following locations. These versions are kept in sync *manually* - there should not be discrepancies, but it is possible.
- On Kaggle: <https://www.kaggle.com/btw78jt/data-conversion-challenge-202004>
- In the GitHub project repo: <https://github.com/A-Breeze/premierconverter>. See the `README.md` for further instructions.

# STOP PRESS
There is a [known issue](#Known-issue) preventing this notebook from running on Windows. Decided not to fix it for the time being, because it will no longer be an issue when we move to using CSVs instead of Excel files.

<!-- This table of contents is updated *manually* -->
# Contents
1. [Setup](#Setup): Import packages, Config variables
1. [Variables](#Variables): Raw data structure, Inputs
1. [Workflow](#Workflow): Load raw data, Remove unwanted extra values, Stem section, Factor sets
1. [Using the functions](#Using-the-functions)
1. [Unused rough work](#Unused-rough-work): Replace multiple string terms, Chained drop a column MultiIndex level

<div align="right" style="text-align: right"><a href="#Contents">Back to Contents</a></div>

# Setup

In [1]:
# Set warning messages
import warnings
# Show all warnings in IPython
warnings.filterwarnings('always')
# Ignore specific numpy warnings (as per <https://github.com/numpy/numpy/issues/11788#issuecomment-422846396>)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
# Other warnings that sometimes occur
warnings.filterwarnings("ignore", message="unclosed file <_io.BufferedReader")

In [2]:
# Determine whether this notebook is running on Kaggle
from pathlib import Path

on_kaggle = False
print("Current working directory: " + str(Path('.').absolute()))
if str(Path('.').absolute()) == '/kaggle/working':
    on_kaggle = True

Current working directory: H:\My Documents\05_Repos\premierconverter\development\compiled


In [3]:
# Import built-in modules
import sys
import platform
import os

# Import external modules
from IPython import __version__ as IPy_version
import numpy as np
import pandas as pd
from openpyxl import __version__ as opyxl_version
from openpyxl import load_workbook
from click import __version__ as click_version

# Import project modules
if not on_kaggle:
    from pyprojroot import here
    root_dir_path = here()
    # Allow modules to be imported relative to the project root directory
    if not sys.path[0] == root_dir_path:
        sys.path.insert(0, str(root_dir_path))
import premierconverter as PCon

# Check they have loaded and the versions are as expected
assert platform.python_version_tuple() == ('3', '6', '6')
print(f"Python version:\t\t\t{sys.version}")
assert IPy_version == '7.13.0'
print(f'IPython version:\t\t{IPy_version}')
assert np.__version__ == '1.18.2'
print(f'numpy version:\t\t\t{np.__version__}')
assert pd.__version__ == '0.25.3'
print(f'pandas version:\t\t\t{pd.__version__}')
assert opyxl_version == '3.0.3'
print(f'openpyxl version:\t\t{opyxl_version}')
assert click_version == '7.1.1'
print(f'click version:\t\t\t{click_version}')
print(f'premierconverter version:\t{PCon.__version__}')

Python version:			3.6.6 |Anaconda, Inc.| (default, Jun 28 2018, 11:27:44) [MSC v.1900 64 bit (AMD64)]
IPython version:		7.13.0
numpy version:			1.18.2
pandas version:			0.25.3
openpyxl version:		3.0.3
click version:			7.1.1
premierconverter version:	0.3.0


In [4]:
# Output exact environment specification, in case it is needed later
print("Capturing full package environment spec")
print("(But note that not all these packages are required)")
!pip freeze > requirements_snapshot.txt
!jupyter --version > jupyter_versions_snapshot.txt

Capturing full package environment spec
(But note that not all these packages are required)


C:\Program Files\Anaconda3-2018.12\envs\premcon_dev_env\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  return process_handler(cmd, _system_body)


C:\Program Files\Anaconda3-2018.12\envs\premcon_dev_env\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  return process_handler(cmd, _system_body)


In [5]:
# Configuration variables
if on_kaggle:
    raw_data_folder_path = Path('/kaggle/input') / 'dummy-premier-data-raw'
else:
    import proj_config
    raw_data_folder_path = proj_config.example_data_dir_path
assert raw_data_folder_path.is_dir()
print("Correct: All locations are available as expected")

Correct: All locations are available as expected


<div align="right" style="text-align: right"><a href="#Contents">Back to Contents</a></div>

# Variables

## Raw data structure

In [6]:
# Configuration variables for the expected format and structure of the data
excel_extensions = ['.xlsx', '.xlsm', '.xltx', '.xltm']  # Note: .xls is *not* readable by openpyxl

raw_struct = {
    'stop_row_at': 'Total Peril Premium',
    'stem': {
        'ncols': 5,
        'chosen_cols': [0,1],
        'col_names': ['Premier_Test_Status', 'Total_Premium'],
        'col_types': [np.dtype('object'), np.dtype('float')],
    },
    'f_set': {
        'ncols': 4,
        'col_names': ['Peril_Factor', 'Relativity', 'Premium_increment', 'Premium_cumulative'],
        'col_types': [np.dtype('object')] + [np.dtype('float')] * 3,
    },
    'bp_name': 'Base Premium',
}

# Output variables, considered to be constants
# Column name of the row IDs
row_id_name = "Ref_num"

## Parameters

In [7]:
# Include Factors which are not found in the data
include_factors = None
if include_factors is None:
    include_factors = []

# Maximum number of rows to read in
nrows = None

# Seperator for Peril_Factor column names in output
pf_sep = '_'

In [8]:
# Input file location
in_filepath = raw_data_folder_path / 'minimal_dummy_data_01.xlsx'
in_sheet = 0

# Checks the file exists and is an Excel file
in_filepath = Path(in_filepath)
if not in_filepath.is_file():
    raise FileNotFoundError(
        "\n\tin_filepath: There is no file at the input location:"
        f"\n\t'{in_filepath.absolute()}'"
        "\n\tCannot read the input data"
    )
if not in_filepath.suffix in excel_extensions:
    raise ValueError(
        f"\n\tin_filepath: The input file extension '{in_filepath.suffix}'"
        f"\n\tis not one of the recognised Excel extensions {excel_extensions}"
    )
print("Correct: Input file exists and has an Excel extension")

Correct: Input file exists and has an Excel extension


In [9]:
# Check the workbook and sheet exists
in_workbook = load_workbook(
    in_filepath,
    read_only=True, data_only=True, keep_links=False
)
if isinstance(in_sheet, int):
    if abs(in_sheet) >= len(in_workbook.worksheets): 
        raise ValueError(
            f"\n\tin_sheet: The sheet number '{in_sheet}' cannot be found"
            f"\n\tin the workbook at location:"
            f"\n\t'{in_filepath.absolute()}'"
        )
    else:
        in_sheet_obj = in_workbook.worksheets[in_sheet]
if isinstance(in_sheet, str):
    if in_sheet not in in_workbook.worksheets:
        raise ValueError(
            f"\n\tin_sheet: The sheet name '{in_sheet}' cannot be found"
            f"\n\tin the workbook at location:"
            f"\n\t'{in_filepath.absolute()}'"
        )
    else:
        in_sheet_obj = in_workbook[in_sheet]
if not (isinstance(in_sheet, int) or isinstance(in_sheet, str)):
    raise ValueError(
        "\n\tin_sheet: Must be a string or integer "
        f"\n\tbut '{in_sheet}' of type '{type(in_sheet).__name__}' was supplied"
    )
print("Correct: Input sheet exists")

Correct: Input sheet exists


In [10]:
# Warn if it is not the expected shape
in_sheet_ncols = in_sheet_obj.max_column
if not (
    # At least the stem columns and one factor set column
    (in_sheet_ncols - 1) >= 
    raw_struct['stem']['ncols'] + raw_struct['f_set']['ncols']
) or not (
    # Stem columns plus a multiple of factor set columns
    (in_sheet_ncols - 1 - raw_struct['stem']['ncols']) 
    % raw_struct['f_set']['ncols'] == 0
):
    warnings.warn(
        f"Raw data: Incorrect number of columns in worksheet: {in_sheet_ncols}"
        "\n\tThere should be: 1 for row ID, "
        f"{raw_struct['stem']['ncols']} for stem section, "
        f"and by a multiple of {raw_struct['f_set']['ncols']} for factor sets"
    )

In [11]:
# Output file location
out_filepath = 'formatted_dummy_data1.xlsx'
out_sheet_name = 'Sheet2'
force_overwrite = False

# Checks
out_filepath = Path(out_filepath)
xl_writer = pd.ExcelWriter(out_filepath, engine = 'openpyxl')

if not out_filepath.parent.is_dir():
    raise FileNotFoundError(
        f"\n\tout_filepath: The folder of the output file does not exist"
        f"Folder path: '{out_filepath.parent}'"
        "\n\tCreate the output folder before running this command"
    )

if out_filepath.is_file():
    out_workbook = load_workbook(out_filepath)
    if out_sheet_name in out_workbook.sheetnames and not force_overwrite:
        raise FileExistsError(
            "\n\tOutput options: Sheet already exists at the output location:"
            f"\n\tLocation: '{out_filepath}'"
            f"\n\tSheet name: '{out_sheet_name}'"
            "\n\tIf you want to overwrite it, re-run with `force_overwrite = True`"
        )
    # Set the pandas ExcelWriter to point at this workbook
    xl_writer.book = out_workbook
    ## ExcelWriter for some reason uses writer.sheets to access the sheet.
    ## If you leave it empty it will not know what sheets are already there
    ## and will create a new sheet. See: <https://stackoverflow.com/a/20221655>
    xl_writer.sheets = dict((ws.title, ws) for ws in out_workbook.worksheets)
else:
    if not out_filepath.suffix in excel_extensions:
        warnings.warn(
            f"out_filepath: The output file extension '{out_filepath.suffix}' "
            "is not a recognised Excel extension",
        )

print("Correct: A suitable location for output has been chosen")

Correct: A suitable location for output has been chosen


<div align="right" style="text-align: right"><a href="#Contents">Back to Contents</a></div>

# Workflow

## Load raw data

In [12]:
df_raw = pd.read_excel(
    in_filepath, sheet_name=in_sheet,
    engine="openpyxl",  # As per: https://stackoverflow.com/a/60709194
    header=None, index_col=0, nrows=nrows,
).rename_axis(index=row_id_name)

df_raw.head()

,1,2,3,4,5,6,7,8,9,10,...,20,21,22,23,24,25,26,27,28,29
Ref_num,,,,,,,,,,,,,,,,,,,,,
1,Ok,96.95,NaN,NaN,9,Peril1 Base Premium,0.000000,91.95,91.95,AnotherPrlBase Premium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ok,170.73,NaN,NaN,11,AnotherPrlBase Premium,0.000000,101.56,101.56,AnotherPrlFactor1,...,-37.97,70.18,Total Peril Premium,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Error: Data Server Text Value Not Found (1111)...,0.00,NaN,NaN,9,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ok,161.68,NaN,NaN,5,Peril1NewFact,0.999998,0.00,110.34,Peril1Factor1,...,51.34,51.34,Peril1 Base Premium,0.0,91.95,91.95,Total Peril Premium,NaN,NaN,NaN


## Remove unwanted extra values

In [13]:
def set_na_after_val(row_sers, match_val):
    """
    Return a copy of `row_sers` with values on or after the 
    first instance of `match_val` set to NaN (i.e. missing).
    
    row_sers: Series to look through
    match_val: Scalar to find. If no occurrences are found, 
        return a copy of the original Serires.
    """
    res = row_sers.to_frame('val').assign(
        keep=lambda df: pd.Series(np.select(
            # All matching values are set to 1.0
            # Others are set to NaN
            [df['val'] == match_val],
            [1.0],
            default=np.nan,
        ), index=df.index).ffill(
            # Forward fill so that all entries on or after the first
            # match are set to 1.0, not NaN
        ).isna(),  # Convert NaN/1.0 to True/False
        # Take the original value, except where 'keep' is False,
        # where the value is replaced with NaN.
        new_val=lambda df: df['val'].where(df['keep'], np.nan)
    )['new_val']
    return(res)

In [14]:
def trim_na_cols(df):
    """
    Remove any columns on the right of a DataFrame `df` which have all missing 
    values up to the first column with at least one non-missing value.
    """
    keep_col = df.isna().mean(
        # Get proportion of each column that is missing.
        # Columns with all missing values will have 1.0 proportion missing.
    ).to_frame('prop_missing').assign(
        keep=lambda df: pd.Series(np.select(
            # All columns with at least one non-missing value are set to 1.0
            # Others are set to NaN
            [df['prop_missing'] < 1.],
            [1.0],
            default=np.nan,
        ), index=df.index).bfill(
            # Backward fill so that all columns on or before the last
            # column with at least one non-missing value are set to 1.0
        ).notna()  # Convert 1.0/NaN to True/False
    )['keep']
    return(df.loc[:, keep_col])

In [15]:
# Set unwanted values to NaN
# and remove surplus columns (with all missing values) from the right
df_trimmed = df_raw.apply(
    set_na_after_val, match_val=raw_struct['stop_row_at'], axis=1
).pipe(trim_na_cols)

df_trimmed.head()

,1,2,3,4,5,6,7,8,9,10,...,16,17,18,19,20,21,22,23,24,25
Ref_num,,,,,,,,,,,,,,,,,,,,,
1,Ok,96.95,NaN,NaN,9,Peril1 Base Premium,0.000000,91.95,91.95,AnotherPrlBase Premium,...,-0.17,91.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ok,170.73,NaN,NaN,11,AnotherPrlBase Premium,0.000000,101.56,101.56,AnotherPrlFactor1,...,100.55,100.55,AnotherPrlSomeFact,0.648875,-37.97,70.18,NaN,NaN,NaN,NaN
3,Error: Data Server Text Value Not Found (1111)...,0.00,NaN,NaN,9,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ok,161.68,NaN,NaN,5,Peril1NewFact,0.999998,0.00,110.34,Peril1Factor1,...,NaN,NaN,AnotherPrlBase Premium,0.000000,51.34,51.34,Peril1 Base Premium,0.0,91.95,91.95


In [16]:
# Check it is as expected
if not (
    # At least the stem columns and one factor set column
    df_trimmed.shape[1] >= 
    raw_struct['stem']['ncols'] + 1 * raw_struct['f_set']['ncols']
) or not (
    # Stem columns plus a multiple of factor set columns
    (df_trimmed.shape[1] - raw_struct['stem']['ncols']) 
    % raw_struct['f_set']['ncols'] == 0
):
    warnings.warn(
        f"Trimmed data: Incorrect number of columns with relevant data: {df_trimmed.shape[1] + 1}"
        "\n\tThere should be: 1 for index, "
        f"{raw_struct['stem']['ncols']} for stem section, "
        f"and by a multiple of {raw_struct['f_set']['ncols']} for factor sets"
    )

## Stem section

In [17]:
# Get the stem section of columns
df_stem = df_trimmed.iloc[
    :, raw_struct['stem']['chosen_cols']
].pipe(  # Rename the columns
    lambda df: df.rename(columns=dict(zip(
        df.columns, 
        raw_struct['stem']['col_names']
    )))
)

df_stem.head()

,Premier_Test_Status,Total_Premium
Ref_num,,
1,Ok,96.95
2,Ok,170.73
3,Error: Data Server Text Value Not Found (1111)...,0.00
4,Ok,161.68


In [18]:
# Checks
if not (
    df_stem.dtypes == raw_struct['stem']['col_types']
).all():
    warnings.warn(
        "Stem columns: Unexpected column data types"
        f"\n\tExepcted: {raw_struct['stem']['col_types']}"
        f"\n\tActual:   {df_stem.dtypes.tolist()}"
    )

## Factor sets

In [19]:
# Combine the rest of the DataFrame into one
df_fsets = pd.concat([
    # For each of the factor sets of columns
    df_trimmed.iloc[  # Select the columns
        :, fset_start_col:(fset_start_col + raw_struct['f_set']['ncols'])
    ].dropna(  # Remove rows that have all missing values
        how="all"
    ).pipe(lambda df: df.rename(columns=dict(zip(  # Rename columns
        df.columns, raw_struct['f_set']['col_names']
    )))).reset_index()  # Get row_ID as a column

    for fset_start_col in range(
        raw_struct['stem']['ncols'], df_trimmed.shape[1], raw_struct['f_set']['ncols']
    )
], sort=False)

df_fsets.head()

,Ref_num,Peril_Factor,Relativity,Premium_increment,Premium_cumulative
0,1,Peril1 Base Premium,0.000000,91.95,91.95
1,2,AnotherPrlBase Premium,0.000000,101.56,101.56
2,4,Peril1NewFact,0.999998,0.00,110.34
0,1,AnotherPrlBase Premium,0.000000,5.17,5.17
1,2,AnotherPrlFactor1,1.064887,6.59,108.15


In [20]:
# Checks
if not (
    df_fsets[raw_struct['f_set']['col_names']].dtypes == 
    raw_struct['f_set']['col_types']
).all():
    warnings.warn(
        "Factor sets columns: Unexpected column data types"
        f"\n\tExepcted: {raw_struct['f_set']['col_types']}"
        f"\n\tActual:   {df_fsets[raw_struct['f_set']['col_names']].dtypes.tolist()}"
    )

In [21]:
perils_implied = df_fsets.Peril_Factor.drop_duplicates(  # Get only unique 'Peril_Factor' combinations
).to_frame().pipe(lambda df: df.loc[  # Filter to leave only 'Base Premium' occurences
    df.Peril_Factor.str.contains(raw_struct['bp_name']), :
]).assign(
    # Get the 'Peril' part of 'Peril_Factor'
    Peril=lambda df: df.Peril_Factor.str.replace(raw_struct['bp_name'], "").str.strip()
).Peril.sort_values().to_list()

perils_implied

['AnotherPrl', 'Peril1']

In [22]:
# Check that every 'Peril_Factor' starts with a Peril
if not df_fsets.Peril_Factor.str.startswith(
    tuple(perils_implied)
).all():
    warnings.warn(
        "Implied perils: Not every Peril_Factor starts with a Peril. "
        "Suggests the raw data format is not as expected."
    )
if '' in perils_implied:
    warnings.warn(
        "Implied perils: Empty string has been implied. "
        "Suggests the raw data format is not as expected."
    )

In [23]:
# Split out Peril_Factor
df_fsets_split = df_fsets.assign(
    # Split the Peril_Factor column into two
    Factor=lambda df: df.Peril_Factor.str.replace(
            '|'.join(perils_implied), ""
    ).str.strip(),
    Peril=lambda df: df.apply(
        lambda row: row.Peril_Factor.replace(row.Factor, "").strip()
        , axis=1
    )
).drop(columns='Peril_Factor')

df_fsets_split.head()

,Ref_num,Relativity,Premium_increment,Premium_cumulative,Factor,Peril
0,1,0.000000,91.95,91.95,Base Premium,Peril1
1,2,0.000000,101.56,101.56,Base Premium,AnotherPrl
2,4,0.999998,0.00,110.34,NewFact,Peril1
0,1,0.000000,5.17,5.17,Base Premium,AnotherPrl
1,2,1.064887,6.59,108.15,Factor1,AnotherPrl


In [24]:
# Get the Base Premiums for all row_IDs and Perils
df_base_prems = df_fsets_split.query(
    # Get only the Base Preimum rows
    f"Factor == '{raw_struct['bp_name']}'"
).assign(
    # Create Peril_Factor combination for column names
    Peril_Factor=lambda df: df.Peril + pf_sep + df.Factor,
    Custom_order=0,  # Will be used later to ensure desired column order
).pivot_table(
    # Pivot to 'Peril_Factor' columns and one row per row_ID
    index=row_id_name,
    columns=['Peril', 'Custom_order', 'Peril_Factor'],
    values='Premium_cumulative'
)

df_base_prems.head()

Peril,AnotherPrl,Peril1
Custom_order,0,0
Peril_Factor,AnotherPrl_Base Premium,Peril1_Base Premium
Ref_num,,
1,5.17,91.95
2,101.56,100.55
4,51.34,91.95


In [25]:
# Warning if the data set is not complete
if df_base_prems.isna().sum().sum() > 0:
    warnings.warn(
        "Base Premiums: Base Premium is missing for some rows and Perils."
        "Suggests the raw data format is not as expected."
    )

In [26]:
# Ensure every row_ID has a row for every Peril, Factor combination
# Get the Relativity for all row_ID, Perils and Factors
df_factors = df_fsets_split.query(
    # Get only the Factor rows
    f"Factor != '{raw_struct['bp_name']}'"
).drop(
    columns=['Premium_increment', 'Premium_cumulative']
).set_index(
    # Ensure there is one row for every combination of row_ID, Peril, Factor
    [row_id_name, 'Peril', 'Factor']
).pipe(lambda df: df.reindex(index=pd.MultiIndex.from_product([
    df.index.get_level_values(row_id_name).unique(),
    df.index.get_level_values('Peril').unique(),
    # Include additional factors if desired from the inputs
    set(df.index.get_level_values('Factor').tolist() + include_factors),
], names = df.index.names
))).sort_index().fillna({  # Any new rows need to have Relativity of 1
    'Relativity': 1.,
}).reset_index().assign(
    # Create Peril_Factor combination for column names
    Peril_Factor=lambda df: df.Peril + pf_sep + df.Factor,
    Custom_order=1
).pivot_table(
    # Pivot to 'Peril_Factor' columns and one row per row_ID
    index=row_id_name,
    columns=['Peril', 'Custom_order', 'Peril_Factor'],
    values='Relativity'
)

df_factors.head()

Peril                AnotherPrl                                         \
Custom_order                  1                                          
Peril_Factor AnotherPrl_Factor1 AnotherPrl_NewFact AnotherPrl_SomeFact   
Ref_num                                                                  
1                      1.000000                1.0            1.000000   
2                      1.064887                1.0            0.648875   
4                      1.000000                1.0            1.000000   

Peril                Peril1                                 
Custom_order              1                                 
Peril_Factor Peril1_Factor1 Peril1_NewFact Peril1_SomeFact  
Ref_num                                                     
1                   0.99818       1.000000             1.0  
2                   1.00000       1.000000             1.0  
4                   1.20000       0.999998             1.0

In [27]:
# Checks
if not df_factors.apply(lambda col: (col > 0)).all().all():
    warnings.warn(
        "Factor relativities: At least one relativity is below zero."
    )

In [28]:
# Combine Base Premium and Factors columns
df_base_factors = df_base_prems.merge(
    df_factors,
    how='inner', left_index=True, right_index=True
).pipe(
    # Sort columns (uses 'Custom_order')
    lambda df: df[df.columns.sort_values()]
)

# Drop unwanted levels of the column MultiIndex
# Possible to do this following in a chain, but much to complicated
# See 'Chained drop a column MultiIndex level' in 'Unused rough work'
df_base_factors.columns = df_base_factors.columns.get_level_values('Peril_Factor')

df_base_factors.head()

Peril_Factor,AnotherPrl_Base Premium,AnotherPrl_Factor1,AnotherPrl_NewFact,AnotherPrl_SomeFact,Peril1_Base Premium,Peril1_Factor1,Peril1_NewFact,Peril1_SomeFact
Ref_num,,,,,,,,
1,5.17,1.000000,1.0,1.000000,91.95,0.99818,1.000000,1.0
2,101.56,1.064887,1.0,0.648875,100.55,1.00000,1.000000,1.0
4,51.34,1.000000,1.0,1.000000,91.95,1.20000,0.999998,1.0


In [29]:
# Join back on to stem section
df_formatted = df_stem.merge(
    df_base_factors,
    how='left', left_index=True, right_index=True
).fillna(0.)  # The only mising values are from 'error' rows

df_formatted.iloc[:10,:20]

,Premier_Test_Status,Total_Premium,AnotherPrl_Base Premium,AnotherPrl_Factor1,AnotherPrl_NewFact,AnotherPrl_SomeFact,Peril1_Base Premium,Peril1_Factor1,Peril1_NewFact,Peril1_SomeFact
Ref_num,,,,,,,,,,
1,Ok,96.95,5.17,1.000000,1.0,1.000000,91.95,0.99818,1.000000,1.0
2,Ok,170.73,101.56,1.064887,1.0,0.648875,100.55,1.00000,1.000000,1.0
3,Error: Data Server Text Value Not Found (1111)...,0.00,0.00,0.000000,0.0,0.000000,0.00,0.00000,0.000000,0.0
4,Ok,161.68,51.34,1.000000,1.0,1.000000,91.95,1.20000,0.999998,1.0


## Output to Excel

In [30]:
# Save it
df_formatted.to_excel(xl_writer, sheet_name=out_sheet_name)
xl_writer.save()
xl_writer.close()
print("Output saved")

Output saved


### Reload the spreadsheet to check it worked

In [31]:
# Check it worked
df_reload = pd.read_excel(
    out_filepath, sheet_name=out_sheet_name,
    engine="openpyxl",  # As per: https://stackoverflow.com/a/60709194
    index_col=[0],
).apply(lambda col: (
    col if col.name in raw_struct['stem']['col_names'][0]
    else col.astype('float')
))

df_reload.head()

,Premier_Test_Status,Total_Premium,AnotherPrl_Base Premium,AnotherPrl_Factor1,AnotherPrl_NewFact,AnotherPrl_SomeFact,Peril1_Base Premium,Peril1_Factor1,Peril1_NewFact,Peril1_SomeFact
Ref_num,,,,,,,,,,
1,Ok,96.95,5.17,1.000000,1.0,1.000000,91.95,0.99818,1.000000,1.0
2,Ok,170.73,101.56,1.064887,1.0,0.648875,100.55,1.00000,1.000000,1.0
3,Error: Data Server Text Value Not Found (1111)...,0.00,0.00,0.000000,0.0,0.000000,0.00,0.00000,0.000000,0.0
4,Ok,161.68,51.34,1.000000,1.0,1.000000,91.95,1.20000,0.999998,1.0


In [32]:
assert (df_formatted.dtypes == df_reload.dtypes).all()
assert df_reload.shape == df_formatted.shape
assert (df_formatted.index == df_reload.index).all()
assert df_formatted.iloc[:,1:].apply(
    lambda col: np.abs(col - df_reload[col.name]) < 1e-10
).all().all()
print("Correct: The reloaded values are equal, up to floating point tolerance")

Correct: The reloaded values are equal, up to floating point tolerance


## Load expected output to check it is as expected

In [33]:
# Location of sheet of expected results
expected_filepath = raw_data_folder_path / 'minimal_dummy_data_01.xlsx'
expected_sheet = 'expected_result'

In [34]:
# Check it worked
df_expected = pd.read_excel(
    expected_filepath, sheet_name=expected_sheet,
    engine="openpyxl",
    index_col=[0],
).apply(lambda col: (
    col if col.name in raw_struct['stem']['col_names'][0]
    else col.astype('float')
))

df_expected.head()

,Premier_Test_Status,Total_Premium,AnotherPrl_Base Premium,AnotherPrl_Factor1,AnotherPrl_NewFact,AnotherPrl_SomeFact,Peril1_Base Premium,Peril1_Factor1,Peril1_NewFact,Peril1_SomeFact
Ref_num,,,,,,,,,,
1,Ok,96.95,5.17,1.000000,1.0,1.000000,91.95,0.99818,1.000000,1.0
2,Ok,170.73,101.56,1.064887,1.0,0.648875,100.55,1.00000,1.000000,1.0
3,Error: Data Server Text Value Not Found (1111)...,0.00,0.00,0.000000,0.0,0.000000,0.00,0.00000,0.000000,0.0
4,Ok,161.68,51.34,1.000000,1.0,1.000000,91.95,1.20000,0.999998,1.0


In [35]:
assert (df_formatted.dtypes == df_expected.dtypes).all()
assert df_expected.shape == df_formatted.shape
assert (df_formatted.index == df_expected.index).all()
assert df_formatted.iloc[:,1:].apply(
    lambda col: np.abs(col - df_expected[col.name]) < 1e-10
).all().all()
print("Correct: The reloaded values are equal, up to floating point tolerance")

Correct: The reloaded values are equal, up to floating point tolerance


<div align="right" style="text-align: right"><a href="#Contents">Back to Contents</a></div>

# Using the functions

In [36]:
help(PCon.convert)

Help on function convert in module premierconverter:

convert(in_filepath, out_filepath, in_sheet=0, out_sheet_name='Sheet1', force_overwrite=False, nrows=None, **kwargs)
    Load raw data from Excel, convert to specified format, and save result
    
    in_filepath: Path to Excel file containing a sheet with the raw data
    out_filepath: Path of an Excel file to save the formatted data
        If it does not exist, a new workbook will be created.
        The directory must already exist.
    
    in_sheet: Sheet number (starting from 0) or sheet name to read
    out_sheet_name: Name of the sheet to save the formatted data
    force_overwrite: Set to True if you want to overwrite the existing workbook sheet
    
    nrows: Maximum number of rows to read
    **kwargs: Other arguments to pass to convert_df
    
    Returns: (out_filepath, out_sheet_name) if it completes



In [37]:
# Run with default arguments
in_filepath = raw_data_folder_path / 'minimal_dummy_data_01.xlsx'
out_filepath = 'formatted_data.xlsx'
res_filepath, res_sheet_name = PCon.convert(in_filepath, out_filepath)

Output saved
File:	H:\My Documents\05_Repos\premierconverter\development\compiled\formatted_data.xlsx
Sheet:	Sheet1


In [38]:
# Run the pipeline manually to check
# Load raw data
df_raw = PCon.read_raw_data(in_filepath)
# Get converted DataFrame
df_formatted = PCon.convert_df(df_raw)

df_formatted.head()

,Premier_Test_Status,Total_Premium,AnotherPrl_Base Premium,AnotherPrl_Factor1,AnotherPrl_NewFact,AnotherPrl_SomeFact,Peril1_Base Premium,Peril1_Factor1,Peril1_NewFact,Peril1_SomeFact
Ref_num,,,,,,,,,,
1,Ok,96.95,5.17,1.000000,1.0,1.000000,91.95,0.99818,1.000000,1.0
2,Ok,170.73,101.56,1.064887,1.0,0.648875,100.55,1.00000,1.000000,1.0
3,Error: Data Server Text Value Not Found (1111)...,0.00,0.00,0.000000,0.0,0.000000,0.00,0.00000,0.000000,0.0
4,Ok,161.68,51.34,1.000000,1.0,1.000000,91.95,1.20000,0.999998,1.0


### Known issue
On Windows, the following `pd.read_excel`  using `openpyxl` is opening the file, but not closing the stream. This means the file is then locked for changes (e.g. deletion, later on in the script). As per: <https://github.com/pandas-dev/pandas/issues/29803> (not fixed at the time of writing). 

Seeing as the next development step is to change from using Excel files to CSVs, I have not fixed this issue.

In [39]:
# Reload resulting data from workbook
df_reload = PCon.load_formatted_spreadsheet(res_filepath, res_sheet_name)

# Check it matches expectations
if PCon.formatted_dfs_are_equal(df_formatted, df_reload):
    print("Correct: The reloaded values are equal, up to floating point tolerance")

Correct: The reloaded values are equal, up to floating point tolerance


In [40]:
# Check against expected output from manually created worksheet
expected_filepath = raw_data_folder_path / 'minimal_dummy_data_01.xlsx'
expected_sheet = 'expected_result'
df_expected = PCon.load_formatted_spreadsheet(expected_filepath, expected_sheet)

# Check it matches expectations
if PCon.formatted_dfs_are_equal(df_formatted, df_expected):
    print("Correct: The reloaded values are equal, up to floating point tolerance")

Correct: The reloaded values are equal, up to floating point tolerance


The following will fail due to a [known issue](#Known-issue).

In [41]:
# Delete the results workbook
try: 
    res_filepath.unlink()
    print("Workspace restored")
except PermissionError:
    print("File deletion has FAILED due to a known error")

File deletion has FAILED due to a known error


<div align="right" style="text-align: right"><a href="#Contents">Back to Contents</a></div>

# Unused rough work

## Replace multiple string terms

In [42]:
import functools

def multi_replace(base_str, replacement_dict):
    """
    Run str.replace() multiple times to replace multiple terms.
    
    base_str: Starting string from which you want to replace substrings
    replacement_dict: Each item of the dictionary is {string_to_replace: replacement_string}
    """
    return(functools.reduce(
        lambda current_str, replace_pair: current_str.replace(*replace_pair),
        {key: str(val) for key, val in replacement_dict.items()}.items(),
        base_str
    ))

## Chained drop a column MultiIndex level

In [43]:
# df_base_factors = df_base_prems.merge(
#     df_factors,
#     how='inner', left_index=True, right_index=True
# ).pipe(
#     lambda df: df[df.columns.sort_values()]
# ).rename(
#     columns=lambda x: '', level = 'Peril'
# ).rename(
#     columns=lambda x: '', level = 'Custom_order'
# ).stack([0,1])#.reset_index(level=[1,2], drop=True)

# df_base_factors.head()

<div align="right" style="text-align: right"><a href="#Contents">Back to Contents</a></div>